<a href="https://colab.research.google.com/github/AK18k/ex3/blob/main/Ex3_3_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/AK18k/ex3


Cloning into 'ex3'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 28 (delta 12), reused 8 (delta 0), pack-reused 0
Unpacking objects: 100% (28/28), 29.46 MiB | 8.37 MiB/s, done.


In [3]:
import os
from google.colab import drive
# drive.mount('/content/drive')
DATA_PATH = '/content/drive/ex3/data'
PATH = '/content/drive/ex3'


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.nn.modules.activation import Softplus
from torch.utils.data import Dataset, DataLoader

#from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC


# Define hyperparameters
input_size = 28 * 28  # Size of the input images (28x28 pixels)
latent_size = 50  # Length of the latent vector
VAE_batch_size = 64
SVM_batch_size = 64
VAE_epochs = 3
SVM_epochs = 10
learning_rate = 1e-3
num_hidden_units = 600
num_of_labeled_samples = 100
expansion_rate = 10


# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(42)

In [5]:
# Load FashionMNIST dataset

transform = transforms.ToTensor()

train_dataset = datasets.FashionMNIST(root='data', train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root=DATA_PATH, train=False, transform=transform, download=True)

100%|██████████| 26421880/26421880 [00:03<00:00, 8230995.95it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 141058.30it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 2594393.04it/s]


...

Extracting /content/drive/ex3/data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /content/drive/ex3/data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 2605863.88it/s]


Extracting /content/drive/ex3/data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /content/drive/ex3/data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 6724471.19it/s]

Extracting /content/drive/ex3/data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /content/drive/ex3/data/FashionMNIST/raw



In [6]:
def split_to_labeled(original_dataset, num_of_labeled_samples):
  class_count = {}
  for _, label in original_dataset:
      if label in class_count:
          class_count[label] += 1
      else:
          class_count[label] = 1

  # Calculate the desired number of samples for each class in the new dataset
  samples_per_class = num_of_labeled_samples // len(class_count)

  # Create a list to store the selected samples
  selected_samples = []

  # Iterate through the original dataset and select the desired number of samples from each class
  selected_count = {label: 0 for label in class_count.keys()}
  for data, label in original_dataset:
      if selected_count[label] < samples_per_class:
          selected_samples.append((data, label))
          selected_count[label] += 1

  class NewDataset(Dataset):
    def __init__(self, samples):
        self.data = [data for data, _ in samples]
        self.labels = [label for _, label in samples]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.labels[index]

  # Create an instance of the new dataset using the selected samples
  labled_dataset = NewDataset(selected_samples)

  return labled_dataset

def reparameterize(mu, logvar):
    std = torch.exp(0.5 * logvar)
    eps = torch.randn_like(std)
    z = mu + eps * std
    return z


In [7]:
#####################################################################
# Create and train the VAE model
# Input:
#   - data_loader - a dataloader with images and labels
# Output:
#   - the VAE model
#####################################################################


# Define the VAE architecture
class VAE(nn.Module):
    def __init__(self, input_size, latent_size):
        super(VAE, self).__init__()

        # Encoder layers
        self.encoder = nn.Sequential(
            nn.Linear(input_size, num_hidden_units),
            nn.Softplus(),
            nn.Linear(num_hidden_units, latent_size * 2)  # Output mu and logvar for each latent dimension
        )

        # Decoder layers
        self.decoder = nn.Sequential(
            nn.Linear(latent_size, num_hidden_units),
            nn.Softplus(),
            nn.Linear(num_hidden_units, input_size),
            nn.Sigmoid()  # Output values between 0 and 1
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        z = mu + eps * std
        return z

    def forward(self, x):
        # Encoder
        encoded = self.encoder(x)
        mu, logvar = torch.chunk(encoded, 2, dim=1)
        z = self.reparameterize(mu, logvar)

        # Decoder
        reconstructed = self.decoder(z)
        return reconstructed, mu, logvar


def train_VAE(data_loader):
  # Create VAE model
  VAE_model = VAE(input_size, latent_size).to(device)

  # Define loss function
  criterion = nn.BCELoss(reduction='sum')  # Binary cross-entropy loss

  # Define optimizer
  optimizer = optim.Adam(VAE_model.parameters(), lr=learning_rate)

  # Training loop
  for epoch in range(VAE_epochs):
      for i, (images, _) in enumerate(data_loader):
          # Flatten input images
          images = images.view(images.size(0), -1).to(device)

          # Forward pass
          reconstructed, mu, logvar = VAE_model(images)

          # Compute reconstruction loss and KL divergence
          reconstruction_loss = criterion(reconstructed, images)
          kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

          # Total loss
          loss = reconstruction_loss + kl_divergence

          # Backward and optimize
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          if (i+1) % 100 == 0:
              print(f"VAE train Epoch [{epoch+1}/{VAE_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

  torch.save(VAE_model.state_dict(), 'VAE_model.pth')
  print('model saved')

  return VAE_model



In [8]:
###############################################################
# Passes a dataset of images through a pretrained VAE model
# Inputs:
#   - VAE_model
#   - data - the dataset (not a dataloader)
# Output:
#   - output_vectors - latent vectors
###############################################################

def images_to_latent(VAE_model, data):
  #data_loader = DataLoader(data, shuffle=False) # Ofer removed batch_size=VAE_batch_size because there is no training

    # Set the model to evaluation mode
  VAE_model.eval()

  # Create an empty list to store the output vectors
  z_output_vectors = []
  mu_output_vectors = []
  logvar_output_vectors = []
  z_output_vectors.to(device)
  mu_output_vectors.to(device)
  logvar_output_vectors.to(device)
  # Pass the dataset through the VAE model
  with torch.no_grad():
      for images, _ in data_loader:
          images = images.to(device)
          # Obtain the output vectors from the VAE model
          z, mu, logvar = VAE_model(data_loader)
          z_output_vectors.append(z)
          mu_output_vectors.append(mu)
          logvar_output_vectors.append(logvar)

  # Concatenate the output vectors into a single tensor
  output_vectors = torch.cat(output_vectors, dim=0)

  return z_output_vectors, mu_output_vectors, logvar_output_vectors

In [9]:

def SVM_model(kernel='poly', degree=3):
  SVM_model = SVC(kernel=kernel, degree=degree)
  return SVM_model

def train_SVM(SVM_model, train_dataset):
  data = train_dataset.data
  data = data.cpu()
  data_np = data.detach().numpy()

  label = train_dataset.targets
  label = label.cpu()
  label_np = label.detach().numpy()
  SVM_model.fit(data_np, label_np)

  return SVM_model

def test_SVM(SVM_model, test_dataset):
  data = test_dataset.data
  data = data.cpu()
  data_np = data.detach().numpy()

  label = test_dataset.targets
  label = label.cpu()
  label_np = label.detach().numpy()
  accuracy = SVM_model.score(data_np, label_np)
  return accuracy






In [10]:
def reparameterize(mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        z = mu + eps * std
        return z

In [23]:
train_loader = DataLoader(train_dataset, batch_size=VAE_batch_size, shuffle=True)
VAE_model = train_VAE(train_loader)








VAE train Epoch [1/3], Step [100/938], Loss: 18654.8789
VAE train Epoch [1/3], Step [200/938], Loss: 19123.7988
VAE train Epoch [1/3], Step [300/938], Loss: 17301.4824
VAE train Epoch [1/3], Step [400/938], Loss: 17060.1699
VAE train Epoch [1/3], Step [500/938], Loss: 18406.0723
VAE train Epoch [1/3], Step [600/938], Loss: 16799.8047
VAE train Epoch [1/3], Step [700/938], Loss: 16766.6582
VAE train Epoch [1/3], Step [800/938], Loss: 16517.6582
VAE train Epoch [1/3], Step [900/938], Loss: 16050.0830
VAE train Epoch [2/3], Step [100/938], Loss: 15607.9961
VAE train Epoch [2/3], Step [200/938], Loss: 16278.2148
VAE train Epoch [2/3], Step [300/938], Loss: 16359.7520
VAE train Epoch [2/3], Step [400/938], Loss: 16190.3535
VAE train Epoch [2/3], Step [500/938], Loss: 15984.1348
VAE train Epoch [2/3], Step [600/938], Loss: 14829.5420
VAE train Epoch [2/3], Step [700/938], Loss: 15757.3506
VAE train Epoch [2/3], Step [800/938], Loss: 15931.0830
VAE train Epoch [2/3], Step [900/938], Loss: 162

In [37]:
# Split the train dataset
print(f'Spliting the training dataset to {num_of_labeled_samples} labled samples')
labled_dataset = split_to_labeled(train_dataset, num_of_labeled_samples)

z_data = torch.zeros([0,50]).to(device)
z_label = torch.zeros([0]).to(device)

expansion_rate = 15

#create the expanded latent vector space
print('Learning mu and logvar from labeled dataset')
data_loader = DataLoader(labled_dataset, batch_size=VAE_batch_size, shuffle=False)
for batch in data_loader:
    images, _ = batch  # Assuming you don't need the labels
    print(f'batch.shape={images.size()}')
    images = images.view(images.size(0), -1).to(device)
    _, mu, logvar = VAE_model(images)

    print(f'Expanding the labeled dataset by {expansion_rate}')
    for i in range(0,len(mu)):
      for e in range(0,expansion_rate):
        new_z = reparameterize(mu[i], logvar[i])
        new_z = (new_z.unsqueeze(0))
        new_label = torch.tensor(labled_dataset.labels[i]).to(device)
        new_label = (new_label.unsqueeze(0))
        z_data = torch.cat((z_data, new_z), 0)
        z_label = torch.cat((z_label, new_label), 0).to(torch.int)

print(f'after z_data.shape = {z_data.shape}')
print(f'after z_label.shape = {z_label.shape}')


Spliting the training dataset to 100 labled samples
Learning mu and logvar from labeled dataset
batch.shape=torch.Size([64, 1, 28, 28])
Expanding the labeled dataset by 15
batch.shape=torch.Size([36, 1, 28, 28])
Expanding the labeled dataset by 15
after z_data.shape = torch.Size([1500, 50])
after z_label.shape = torch.Size([1500])


In [38]:
print(f'Total entries to SVM train: {z_data.size()}')

SVM = SVM_model(kernel='poly', degree=3)

print('Training the SVM with expanded latent dataset')
latent_dataset = Dataset()
latent_dataset.data = z_data
latent_dataset.targets = z_label

SVM = train_SVM(SVM, latent_dataset)

Total entries to SVM train: torch.Size([1500, 50])
Training the SVM with expanded latent dataset


In [39]:
data_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
z_data = torch.zeros([0,50]).to(device)
z_label = torch.zeros([0]).to(device)

i = 0
for batch in data_loader:
    images, _ = batch  # Assuming you don't need the labels
    images = images.view(images.size(0), -1).to(device)
    _, mu, logvar = VAE_model(images)
    #mu = m[:,0]
    #logvar = l[:,0]
    new_z = reparameterize(mu, logvar)
    new_z = (new_z.squeeze(1))
    new_label = test_dataset.targets[i].clone().detach().to(device)
    new_label1 = new_label.unsqueeze(0)
    z_data = torch.cat((z_data, new_z), 0)
    z_label = torch.cat((z_label, new_label1), 0).to(torch.int)
    i += 1

result = test_SVM(SVM, latent_dataset)

print(f'Final result of SVM on test dataset: {result}')


Final result of SVM on test dataset: 0.9586666666666667
